In [1]:
# get rid of error messages
import warnings;warnings.simplefilter('ignore')

# set up modules
%pylab inline
rcParams['savefig.dpi'] = 120
%config InlineBackend.figure_format = 'retina'

import math
from scipy.integrate import odeint

import CoolProp
import CoolProp.CoolProp as CP

Populating the interactive namespace from numpy and matplotlib


## Dane

In [2]:
# parametry czynnika
refrigerant = 'R134a'
m_0 = 0.150 # [kg/s]
T_0 = 0 + 273.15 # [K]


# parametry glikolu
T_g1 = 12 + 273.15 # [K] - temperatura cieczy chłodzonej na wlocie do parownika
T_g2 = 5 + 273.15 # [K] - temperatura cieczy chłodzonej na wylocie z parownika

# Średnia logarytmiczna różnica temperatur wymiennika
LMTD = (T_g1 - T_0) - (T_g2 - T_0) / math.log((T_g1 - T_0)/(T_g2 - T_0))

α_g = 800 # [W/m2K]

### Wstępnie założone parametry konstrukcyjne wymiennika płytowego

In [3]:
G = 120e-3 # [m] - szerokość płyt
H = 360e-3 # [m] - wysokość płyt

s_pl = 1.8e-3 # [m] - odległość pomiędzy płytami
δ_pl = 0.4e-3 # [m] - grubość płyt
λ_pl = 20 # [W/mK] - przewodność cieplna płyty ze stali nierdzewnej

β = 70 # kąt wyżłobień na płycie

### Średnica hydrauliczna kanałów
W przypadku wymienników płytowych średnicę hydrauliczną kanału równa jest podwojonej założonej odległości pomiędzy płytami:
$$d_h = 2s_{pl}$$

In [4]:
d_h = 2 * s_pl
print ("* Średnica hydrauliczna kanałów: %.1f [mm]" % (d_h*1e3))

* Średnica hydrauliczna kanałów: 3.6 [mm]


## Obliczenia parametrów płynów

### Czynnik chłodniczy i jego parametry w założonych temperaturach pracy wymiennika
Wszsytkie parametry czynnika chłodniczego w parowniku oblicza się dla średniego stopnia suchości 0.5.

In [5]:
refrigerant = 'R134a'
m_0 = 0.190 # [kg/s]

In [6]:
# ciśnienie nasycenia czynnika chłodniczego
p_n0 = CP.PropsSI('P','Q',0,'T',T_0,refrigerant)
print ("* Ciśnienie nasycenia %s: %.2f [J/kgK]" % (refrigerant,p_n0))

# ciepło właściwe czynnika chłodniczego
c_p0 = CP.PropsSI('C','Q',0,'T',T_0,refrigerant)
print ("* Ciepło właściwe %s: %.2f [J/kgK]" % (refrigerant,c_p0))

# ciepło przemiany fazowej
r_0 = CP.PropsSI('H','Q',1.0,'T',T_0,refrigerant) - CP.PropsSI('H','Q',0,'T',T_0,refrigerant)
print ("* Ciepło przemiany fazowej %s: %.2f [J/kg]" % (refrigerant,r_0))

# gęstość czynnika chłodniczego (ciecz i gaz)
ρ_0l = CP.PropsSI('D','Q',0,'T',T_0,refrigerant)
print ("* Gęstość %s (ciecz): %.2f [kg/m3]" % (refrigerant,ρ_0l))
ρ_0v = CP.PropsSI('D','Q',1,'T',T_0,refrigerant)
print ("* Gęstość %s (gaz): %.2f [kg/m3]" % (refrigerant,ρ_0v))

# lepkość dynamiczna czynnika chłodniczego (ciecz i gaz)
μ_0l = CP.PropsSI('V','Q',0,'T',T_0,refrigerant)
print ("* Lepkość dynamiczna %s (ciecz): %.6f [Pas]" % (refrigerant,μ_0l))
μ_0v = CP.PropsSI('V','Q',1,'T',T_0,refrigerant)
print ("* Lepkość dynamiczna %s (gaz): %.6f [Pas]" % (refrigerant,μ_0v))

# współczynnik przewodzenia ciepła
λ_0 = CP.PropsSI('L','Q',0,'T',T_0,refrigerant)
print ("* Współczynnik przewodzenia ciepła %s: %.3f [W/mK]" % (refrigerant,λ_0))

# napięcie powierzchniowe
σ_0 = CP.PropsSI('I','Q',0,'T',T_0,refrigerant)
print ("* Napięcie powierzchniowe ciepła %s: %.3f [N/m]" % (refrigerant,σ_0))

# Liczba Prandtla
Pr_0 = c_p0 * μ_0l / λ_0
print ("* Liczba Prandtla %s: %.3f [-]" % (refrigerant,Pr_0))

* Ciśnienie nasycenia R134a: 292803.18 [J/kgK]
* Ciepło właściwe R134a: 1341.04 [J/kgK]
* Ciepło przemiany fazowej R134a: 198603.47 [J/kg]
* Gęstość R134a (ciecz): 1294.78 [kg/m3]
* Gęstość R134a (gaz): 14.43 [kg/m3]
* Lepkość dynamiczna R134a (ciecz): 0.000267 [Pas]
* Lepkość dynamiczna R134a (gaz): 0.000011 [Pas]
* Współczynnik przewodzenia ciepła R134a: 0.092 [W/mK]
* Napięcie powierzchniowe ciepła R134a: 0.011 [N/m]
* Liczba Prandtla R134a: 3.884 [-]


### Prędkość przepływu cieczy w kanałach
Prędkość przepływu cieczy w kanałach zależy od tego ile strumieni przepływowywch ma wymiennik. Strumień masy czynnika należy wtedy podzielić stosownie do ilości kanałów.

Prędkość przepływu cieczy w kanałach płyt oblicza się następująco:
$$w = \frac{\dot{m_g}}{G \cdot s_{pl} \cdot \rho_g \cdot n_{zg}}$$
gdzie:
* $G$ - założona szerokość płyt
* $s_{pl}$ - odległość pomiędzy płytami
* $\rho_g$ - gęstość cieczy chłodzonej (dla średniej temperatury medium w parowniku)
* $n_{zg}$ - liczba kanałów w których równolegle przepływa ciecz chłodzona

In [7]:
# Ilość kanałów w których jednocześnie przepływa czynnik chłodniczy
n_z0 = 1

# Strumień masy czynnika w kanale
m_0k = m_0 / n_z0
print ("* Strumień masy czynnika chłodniczego w kanale: %.2f [kg/s]" % (m_0k))

G_0k = m_0k / (s_pl * G)
print ("* Gęstość strumienia masy czynnika chłodniczego w kanale: %.2f [kg/m2s]" % (G_0k))

# Prędkość przepływu czynnika chłodniczego w kanale
w_0k = m_0k  / (G * s_pl * ρ_0v * n_z0)
print ("* Prędkość przepływu czynnika chłodniczego w kanale: %.2f [m/s]" % (w_0k))

* Strumień masy czynnika chłodniczego w kanale: 0.19 [kg/s]
* Gęstość strumienia masy czynnika chłodniczego w kanale: 879.63 [kg/m2s]
* Prędkość przepływu czynnika chłodniczego w kanale: 60.97 [m/s]


### Współczynnik przejmowania ciepła od strony czynnika chłodniczego $\alpha_0$

#### Liczba Reynoldsa czystej cieczy i czystej pary
$$Re_{lo} = \frac{G \cdot d_h}{\mu_l}$$
$$Re_{vo} = \frac{G \cdot d_h}{\mu_v}$$


#### Liczba Reynoldsa frakcji cieczy i frakcji pary
$$Re_l = \frac{G \cdot\left( 1-x\right) \cdot d_h}{\mu_l}$$
$$Re_v = \frac{G \cdot x \cdot d_h}{\mu_l}$$


#### Liczba wrzenia $Bo$
$$Bo = \frac{q}{r G}$$
gdzie:
* $G$ - [kg/m2s] gęstość strumienia masy czynnika chłodniczego 
* $r$ - ciepło przemiany fazowej
* $q$ - gęstość strumienia ciepła

#### Liczba Webera
Liczba Webera jest to stosunek sił inercjalnych do sił napięcia powierzchniowego.
$$We = \frac{G^2 d_h}{\rho_m \sigma}$$
gdzie:
* $\rho$ - [kg/m3] średnia gęstość płynu (pomiędzy cieczą i gazem)
* $G$ - [kg/m2s] gęstość strumienia masy czynnika chłodniczego 
* $d_h$ - [m] wymiar charakterystyczny (średnica hydrauliczna?)
* $\sigma$ - [N/m] napięcie powierzchniowe płynu

#### Liczba Bonda
Wyraża stosunek sił grawitacji do napięcia powierzchniowego.
$$Bd = \frac{\Delta \rho g d_h^2}{\sigma}$$
gdzie:
* $\Delta \rho$ - różnica gęstości ośrodków (a może średnia?)
* $g$ - przyśpieszenie ziemski (we Wrocławiu podobno $g = 9.8115$ - brak źródła)
* $L$ - wymiar charakterystyczny (średnica hydrauliczna)
* $\sigma$ - [N/m] napięcie powierzchniowe płynu

#### Iteracyjne obliczenie wartości współczynnika przejmowania za pomocą korelacji Amalfiego i innych
Źródło: Amalfi et al., 2016, Flow boiling and frictional pressure gradients in plate heat exchangers. Part 2: comparison of literature methods to database and new prediction methods. Int. J. Refrigeration 61, 185-203

Można stosować dla: **R134a, R717, R245fa, R236fa, R600a, R290, R1270, R1234yf, R410A, R507A**.

Dla $Bd<4$:
$$Nu=982 \cdot (\beta/70)^{1.101} \cdot We_m^{0.315} \cdot Bo^{0.320} \cdot \rho^{*-0.224} $$
Dla $Bd>4$
$$Nu = 18.495 \cdot (\beta/70)^{0.248} \cdot Re_v^{0.135} \cdot  Re_{lo}^{0.351} \cdot Bd^{0.235} \cdot Do^{0.198} \cdot \rho^{*-0.223} $$

gdzie $q^* = \rho_{ciecz} / \rho_{gaz}$

In [8]:
q = 100 # wartość początkowa w iteracji
q_old = 0

while abs(q - q_old) > 1e-4:
    q_old = q
    x = 0.5
    Re_0lo = G_0k * (1 - x) * d_h / μ_0l
    Re_0v = G_0k * d_h / μ_0v
        
    print ("q = %.3f [W/m2]" % (q))
    Bo_0 = q / (r_0 * G_0k)
    print ("* Liczba wrzenia: %.6f [-]" % Bo_0)
    
    We_0 = G_0k**2 * d_h / (0.5 * (ρ_0v + ρ_0l) * σ_0 ) #ρ_0v * w_0k**2 * d_h / σ_0
    print ("* Liczba Webera: %.2f [-]" % We_0)
        
    Bd_0 = (ρ_0l - ρ_0v) * 9.8115 * d_h**2 / σ_0
    print ("* Liczba Bonda: %.2f [-]" % Bd_0)

    # dla Bd < 4
    if Bd_0 < 4: Nu_0k = 982 * (β/70)**1.101 * We_0**0.315 * Bo_0**0.320 * (ρ_0l/ρ_0v)**-0.224
    if Bd_0 >= 4: Nu_0k = 18.495 * (β/70)**0.248 * Re_0v**0.135 * Re_0lo**0.351 * Bd_0**0.235 * Bo_0**0.198 * (ρ_0l/ρ_0v)**-0.223
    print ("* Liczba Nusselta: %.2f [-]" % Nu_0k)

    # Współczynnik przejmowania ciepła
    α_0 = Nu_0k * λ_0 / d_h
    print ("* Współczynnik przejmowania ciepła: %.3f [W/m2K]" % α_0)
    
    k_A = 1 / ( (1/α_g) + (δ_pl/λ_pl) + (1/α_0) )
    q = k_A * LMTD

q = 100.000 [W/m2]
* Liczba wrzenia: 0.000001 [-]
* Liczba Webera: 372.37 [-]
* Liczba Bonda: 14.25 [-]
* Liczba Nusselta: 85.07 [-]
* Współczynnik przejmowania ciepła: 2174.355 [W/m2K]
q = 3635.326 [W/m2]
* Liczba wrzenia: 0.000021 [-]
* Liczba Webera: 372.37 [-]
* Liczba Bonda: 14.25 [-]
* Liczba Nusselta: 173.29 [-]
* Współczynnik przejmowania ciepła: 4429.125 [W/m2K]
q = 4204.349 [W/m2]
* Liczba wrzenia: 0.000024 [-]
* Liczba Webera: 372.37 [-]
* Liczba Bonda: 14.25 [-]
* Liczba Nusselta: 178.35 [-]
* Współczynnik przejmowania ciepła: 4558.508 [W/m2K]
q = 4222.439 [W/m2]
* Liczba wrzenia: 0.000024 [-]
* Liczba Webera: 372.37 [-]
* Liczba Bonda: 14.25 [-]
* Liczba Nusselta: 178.50 [-]
* Współczynnik przejmowania ciepła: 4562.385 [W/m2K]
q = 4222.967 [W/m2]
* Liczba wrzenia: 0.000024 [-]
* Liczba Webera: 372.37 [-]
* Liczba Bonda: 14.25 [-]
* Liczba Nusselta: 178.50 [-]
* Współczynnik przejmowania ciepła: 4562.498 [W/m2K]
q = 4222.983 [W/m2]
* Liczba wrzenia: 0.000024 [-]
* Liczba We